# Ejercicio de ejemplo 1

Consider el siguiente problema de programación lineal
Minimizar:
$$f=-x_0+4x_1$$
Sujeto a:
$$-3x_0+x_1\leq 6$$
$$x_0+2x_1 \leq 4$$
$$x_1\geq -3 $$
Donde:
$$- \infty \leq x_0 \leq \infty $$

In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
c=[-1,4]
A=[[-3,1],[1,2]]
b=[6,4]
x0_bounds=(None,None)
x1_bounds=(-3,None)

res =linprog(c,A_ub=A,b_ub=b,bounds=(x0_bounds,x1_bounds),options={"disp":True})
print(res)


Optimization terminated successfully.
         Current function value: -22.000000  
         Iterations: 1
     fun: -22.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([39.,  0.])
  status: 0
 success: True
       x: array([10., -3.])


### Resolviendo con pulp

In [1]:
from pulp import *

In [4]:
prob=LpProblem("Problema 1",LpMinimize)
x0=LpVariable("Variable 1",None,None)
x1=LpVariable("Variable 2",-3,None)

In [5]:
prob += -x0+4*x1 , "Función a minimizar"

In [6]:
prob += -3*x0+x1 <= 6, "Primera restricción"
prob += x0+2*x1 <= 4, "Segunda restricción"

In [7]:
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [8]:
for x in prob.variables():
    print(x.name,"=",x.varValue)

Variable_1 = 10.0
Variable_2 = -3.0


# Ejercicio de ejemplo 2 (inversiones)

El objetivo de este problema es determinar la mejor estrategia de inversión, dados diferentes tipos de bono, la máxima cantidad que puede ser invertida en cada bono , el porcentaje de retorno y los años de madurez.
* Hay una cantidad fija de dinero disponible (750,000)
* Por lo menos la mitad de este dinero debe ser invertido en bonos con 10 años o más para la madurez
* Se puede invertir un máximo del 25$\%$ de esta cantidad en cada bono
* Finalmente, hay otra restricción que no permite usar más de 35$\%$ en bonos de alto riesto

Existen seis opciones de inversión, con las letras correspondientes $A_i$

1. $A_1$: (Tasa de retorno = 8.65$\%$; Años para la madurez=11, Riesgo=Bajo)
2. $A_2$: (Tasa de retorno = 9.50$\%$; Años para la madurez=10, Riesgo=Alto)
3. $A_3$: (Tasa de retorno = 10.00$\%$; Años para la madurez=6, Riesgo=Alto)
4. $A_4$: (Tasa de retorno = 8.75$\%$; Años para la madurez=10, Riesgo=Bajo)
5. $A_5$: (Tasa de retorno = 9.25$\%$; Años para la madurez=7, Riesgo=Alto)
6. $A_6$: (Tasa de retorno = 9.00$\%$; Años para la madurez=13, Riesgo=Bajo)


Lo que se quiere entonces es maximizar el retorno que deja la inversión

### Resolviendo con scipy

Minimizar:
$$8.65A_1+9.50A_2+10A_3+8.75A_4+9.25A_5+9.00A_6 $$
Restricciones:
$$A_1+A_2+A_3+A_4+A_5+A_6 \leq 750000 $$
$$A_1+A_2+A_4+A_6 \geq 750000*.5 $$
$$0< A_1,A_2,A_3,A_4,A_5,A_6 < 750000*.25 $$
$$A_2+A_3+A_5 \leq 750000*.35 $$

In [6]:
import numpy as np
from scipy.optimize import linprog

In [11]:
c=[-8.65,-9.5,-10,-8.75,-9.25,-9] #En este caso al estar maximixando la c va con los valores en negativo
A=[[1,1,1,1,1,1],[-1,-1,0,-1,0,-1],[0,1,1,0,1,0]]
b=[750000,-750000*.5,750000*.35]

In [12]:
Ai_bound=(0,750000*.25)

In [13]:
res =linprog(c,A_ub=A,b_ub=b,bounds=(Ai_bound,Ai_bound,Ai_bound,Ai_bound,Ai_bound,Ai_bound),options={"disp":True})
print(res)

Optimization terminated successfully.
         Current function value: -6888750.000000
         Iterations: 7
     fun: -6888750.0
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([      0.,  187500.,       0.,   75000.,       0.,       0.,
        187500.,       0.,       0.])
  status: 0
 success: True
       x: array([ 112500.,   75000.,  187500.,  187500.,       0.,  187500.])


### Resolviendo con pulp

In [2]:
tipos=["A1","A2","A3","A4","A5","A6"]
tasa_retorno={"A1":8.65,"A2":9.50,"A3":10.00,"A4":8.75,"A5":9.25,"A6":9.00}
años={"A1":11,"A2":10,"A3":6,"A4":10,"A5":7,"A6":13}
Riesgo={"A1":"BAJO","A2":"ALTO","A3":"ALTO","A4":"BAJO","A5":"ALTO","A6":"BAJO"}

In [3]:
prob2=LpProblem("Problema inversion",LpMaximize)

In [4]:
var=LpVariable.dicts("vars",tipos,0,187500)

In [5]:
prob2 += lpSum([tasa_retorno[i]*var[i] for i in tipos]) , "Inversion de retorno total"

In [6]:
#Se agregan las restricciones
prob2 += lpSum([var[i] for i in tipos])<= 750000
const2=[]
for x in tipos :
    if años[x] >= 10:
        const2.append(var[x])
prob2 += lpSum(const2)>=750000*.5
const3=[]
for x in tipos:
    if Riesgo[x]=="ALTO":
        const3.append(var[x])
prob2 += lpSum(const3)<= 750000*.35

In [7]:
prob2.solve()
print("Status:", LpStatus[prob2.status])

Status: Optimal


In [8]:
for x in prob2.variables():
    print(x.name,"=",x.varValue)

vars_A1 = 112500.0
vars_A2 = 75000.0
vars_A3 = 187500.0
vars_A4 = 187500.0
vars_A5 = 0.0
vars_A6 = 187500.0


# Ejercicio de ejemplo 3 (problema de transporte)

Este es un caso curioso, con solo 6 variables (un caso real de problema de transporte puede tener fácilmente más de 1.000 variables) en el cual se aprecia la utilidad de este procedimiento de cálculo.


Existen tres minas de carbón cuya producción diaria es: 


* La mina "a" produce 40 toneladas de carbón por día;
* La mina "b" produce 40 t/día; y,
* La mina "c" produce 20 t/día.

En la zona hay dos centrales termoeléctricas que consumen: 

* La central "d" consume 40 t/día de carbón; y,
* La central "e" consume 60 t/día

Los costos de mercado, de transporte por tonelada son: 

* De "a" a "d" = 2 monedas
* De "a" a "e" = 11 monedas
* De "b" a "d" = 12 monedas
* De "b" a "e" = 24 monedas
* De "c" a "d" = 13 monedas
* De "c" a "e" = 18 monedas

Funcion a minimizar:
$$2ad+11ae+12bd+24be+13cd+18ce$$
Restricciones:
$$ad+ae \leq 40 $$
$$bd+be \leq 40 $$
$$cd+ce \leq 20 $$
$$ad+bd+cd \geq 40 $$
$$ae+be+ce \geq 60 $$

### Resolviendo con scipy

In [14]:
import numpy as np
from scipy.optimize import linprog

In [29]:
c=[2,11,12,24,13,18]
A=[[1,1,0,0,0,0],[0,0,1,1,0,0],[0,0,0,0,1,1],[-1,0,-1,0,-1,0],[0,-1,0,-1,0,-1]]
b=[40,40,20,-40,-60]
ruta_bound=(0,None)

In [30]:
res =linprog(c,A_ub=A,b_ub=b,bounds=(ruta_bound,ruta_bound,ruta_bound,ruta_bound,ruta_bound,ruta_bound),options={"disp":True})
print(res)

Optimization terminated successfully.
         Current function value: 1280.000000 
         Iterations: 10
     fun: 1280.0
 message: 'Optimization terminated successfully.'
     nit: 10
   slack: array([ 0.,  0.,  0.,  0.,  0.])
  status: 0
 success: True
       x: array([  0.,  40.,  40.,   0.,   0.,  20.])


### Resolviendo con pulp

In [23]:
from pulp import *
rutas=["ad","ae","bd","be","cd","ce"] #la variable la definiré como el número de toneladas por ruta
prob3=LpProblem("Minimizar costos",LpMinimize)
ruta=LpVariable.dicts("ruta",rutas,0)

In [24]:
costos={"ad":2,"ae":11,"bd":12,"be":24,"cd":13,"ce":18}

In [25]:
prob3 += lpSum([costos[i]*ruta[i] for i in rutas]), "Minimizar el costo"

In [26]:
#Restricciones 
prob3 += ruta["ad"]+ruta["ae"] <= 40 , "Maxima producción de la mina a"
prob3 += ruta["bd"]+ruta["be"] <= 40 , "Maxima producción de la mina b"
prob3 += ruta["cd"]+ruta["ce"] <= 20 , "Maxima producción de la mina c"
prob3 += ruta["ad"]+ruta["bd"] +ruta["cd"]>= 40 , "Consumo de la fabrica d"
prob3 += ruta["ae"]+ruta["be"] +ruta["ce"]>= 60 , "Consumo de la fabrica e"

In [27]:
prob3.solve()
print("Status:", LpStatus[prob3.status])

Status: Optimal


In [28]:
for x in prob3.variables():
    print(x.name,"=",x.varValue)

ruta_ad = 0.0
ruta_ae = 40.0
ruta_bd = 40.0
ruta_be = 0.0
ruta_cd = 0.0
ruta_ce = 20.0
